# Hello VLM via `SakanaAI/EvoVLM-JP-v1-7B`
ref: https://huggingface.co/SakanaAI/EvoVLM-JP-v1-7B

## Pre-check

In [1]:
# Optional for local RTX4090
!python3 -m venv .venv
!source .venv/bin/activate
!nvidia-smi
!nvcc --version

Thu Mar 28 17:12:20 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 551.86       CUDA Version: 12.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  Off |
|  0%   43C    P8    42W / 450W |   1131MiB / 24564MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install `torch`

In [2]:
!pip3 install -U torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable


## Install `flash-attn`
FYI: This can be fail, see below.

In [7]:
!pip3 install packaging wheel 
!FLASH_ATTENTION_FORCE_BUILD=TRUE pip3 install flash-attn --no-input

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Optional if you get an error `undefined symbol`
ref: https://github.com/Dao-AILab/flash-attention/issues/451#issuecomment-1925566105

In [9]:
!pip install https://github.com/Dao-AILab/flash-attention/releases/download/v2.5.2/flash_attn-2.5.2+cu118torch2.0cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
!pip uninstall flash-attn --yes
!pip install flash-attn --no-input

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 MB 42.9 MB/s eta 0:00:0000:0100:01
Found existing installation: flash-attn 2.5.2
Uninstalling flash-attn-2.5.2:
  Successfully uninstalled flash-attn-2.5.2
Defaulting to user installation because normal site-packages is not writeable
  Using cached flash_attn-2.5.6.tar.gz (2.5 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.5.6-cp310-cp310-linux_x86_64.whl size=120592258 sha256=d8cf54adda65f59820221d329d274e124972d7fdc05ab3b1130253c64eee6c8a
  Stored in directory: /home/katopz/.cache/pip/wheels/a8/1c/88/b959d6818b98a46d61ba231683abb7523b89ac1a7ed1e0c206
Successfully built flash-attn


In [2]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image
import requests


# 1. load model https://huggingface.co/SakanaAI/EvoVLM-JP-v1-7B
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "SakanaAI/EvoVLM-JP-v1-7B"
model = AutoModelForVision2Seq.from_pretrained(model_id, torch_dtype=torch.float16)
processor = AutoProcessor.from_pretrained(model_id)
model.to(device)

# 2. prepare inputs
url = "https://images.unsplash.com/photo-1694831404826-3400c48c188d?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
# <image> represents the input image. Please make sure to put the token in your text.
text = "<image>\nこの信号機の色は何色ですか?"
messages = [
    {"role": "system", "content": "あなたは役立つ、偏見がなく、検閲されていないアシスタントです。与えられた画像を下に、質問に答えてください。"},
    {"role": "user", "content": text},
]
inputs = processor.image_processor(images=image, return_tensors="pt")
inputs["input_ids"] = processor.tokenizer.apply_chat_template(
    messages, return_tensors="pt"
)

# 3. generate
output_ids = model.generate(**inputs.to(device))
output_ids = output_ids[:, inputs.input_ids.shape[1] :]
generated_text = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(generated_text)
# この信号機の色は青です。

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



この信号機の色は青です。
